In [1]:
using Pkg
Pkg.add("Molly")

   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


### Our **first simulation**
#### Fluid acting under a Lennard-Jones potential

In [2]:
using Molly

### Let's define the number of particles and their masses.

In [25]:
n_atoms = 100
atom_mass = 10.0u"u"
atoms = [Atom(mass=atom_mass, σ=0.3u"nm", ϵ=0.2u"kJ * mol^-1") for i in 1:n_atoms]

100-element Vector{Atom{Float64, Quantity{Float64, 𝐌, Unitful.FreeUnits{(u,), 𝐌, nothing}}, Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}, Quantity{Float64, 𝐋² 𝐌 𝐍⁻¹ 𝐓⁻², Unitful.FreeUnits{(kJ, mol⁻¹), 𝐋² 𝐌 𝐍⁻¹ 𝐓⁻², nothing}}}}:
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, ϵ=0.2 kJ mol⁻¹
 ⋮
 Atom with index 1, charge=0.0, mass=10.0 u, σ=0.3 nm, 

Our simulation will run in a cubic box (defined by the ```CubicBoundary```). You can also use a ```TriclinicBoundary```. Simulations in 2 dimensions should use a ```RectangularBoundary```.

In [26]:
boundary = CubicBoundary(2.0u"nm", 2.0u"nm", 2.0u"nm") # Periodic boundary conditions

CubicBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}[2.0 nm, 2.0 nm, 2.0 nm])

In [27]:
pairwise_inters = (LennardJones(),) 

(LennardJones{false, NoCutoff, Int64, Int64, Unitful.FreeUnits{(kJ, nm⁻¹, mol⁻¹), 𝐋 𝐌 𝐍⁻¹ 𝐓⁻², nothing}, Unitful.FreeUnits{(kJ, mol⁻¹), 𝐋² 𝐌 𝐍⁻¹ 𝐓⁻², nothing}}(NoCutoff(), false, true, 1, 1, kJ nm⁻¹ mol⁻¹, kJ mol⁻¹),)

In [28]:
coords = place_atoms(n_atoms, boundary; min_dist=0.3u"nm") # Random placement without clashing

100-element Vector{SVector{3, Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}}}:
 [1.7778077784474735 nm, 0.9901981090002268 nm, 0.4384945043394741 nm]
 [1.935750684649968 nm, 0.2055721474690524 nm, 1.5063566178019583 nm]
 [1.546213357996598 nm, 1.1659426518030647 nm, 1.460299805807513 nm]
 [0.47847879594632126 nm, 1.2990991729112744 nm, 0.0521099467242474 nm]
 [1.4562595128349312 nm, 0.07982685135702217 nm, 0.8255821733668218 nm]
 [0.8662649137457159 nm, 1.859474396658764 nm, 0.663783479630371 nm]
 [0.4793348349081705 nm, 1.7395565471141818 nm, 1.0900456604097102 nm]
 [0.30884067907864954 nm, 1.9223211535292684 nm, 0.659821817368728 nm]
 [1.9960754265938319 nm, 0.34718195736824 nm, 0.1877044374812047 nm]
 [0.8982716368179804 nm, 0.707866969266032 nm, 1.5026839328576662 nm]
 ⋮
 [1.9519778418699143 nm, 1.436206948010006 nm, 0.42787466535532226 nm]
 [1.2825877544825048 nm, 1.7653759946025007 nm, 1.9976633702920252 nm]
 [1.253764556911067 nm, 1.0275276875641701 nm, 1.4310126464

In [29]:
temp = 100.0u"K"

100.0 K

In [30]:
velocities = [random_velocity(atom_mass, temp) for i in 1:n_atoms]

100-element Vector{SVector{3, Quantity{Float64, 𝐋 𝐓⁻¹, Unitful.FreeUnits{(nm, ps⁻¹), 𝐋 𝐓⁻¹, nothing}}}}:
 [-0.12975032175693543 nm ps⁻¹, -0.08175969864693959 nm ps⁻¹, -0.5603977127512478 nm ps⁻¹]
 [-0.13179375392814136 nm ps⁻¹, -0.5707813046023895 nm ps⁻¹, 0.16278177072111755 nm ps⁻¹]
 [0.2072919201082432 nm ps⁻¹, 0.11323359975190546 nm ps⁻¹, 0.27814437233860223 nm ps⁻¹]
 [-0.23170283254292898 nm ps⁻¹, -0.1803082328275613 nm ps⁻¹, -0.22894011997783614 nm ps⁻¹]
 [-0.15069783039297924 nm ps⁻¹, 0.015979133318885726 nm ps⁻¹, -0.38816346557221076 nm ps⁻¹]
 [-0.3262037000997499 nm ps⁻¹, -0.140122735715261 nm ps⁻¹, -0.05891119027699866 nm ps⁻¹]
 [0.09338192517872869 nm ps⁻¹, -0.20120301829562617 nm ps⁻¹, -0.09189805934770477 nm ps⁻¹]
 [-0.38362278839179276 nm ps⁻¹, 0.30281567366233714 nm ps⁻¹, -0.30043115642775153 nm ps⁻¹]
 [0.1447757697424652 nm ps⁻¹, 0.006264978077395682 nm ps⁻¹, 0.01602893854617699 nm ps⁻¹]
 [-0.5501913904260566 nm ps⁻¹, 0.2097223413757074 nm ps⁻¹, 0.07853052252982931 nm p

In [31]:
sys = System(
    atoms=atoms,
    pairwise_inters=pairwise_inters,
    coords=coords,
    velocities=velocities,
    boundary=boundary,
    loggers=(
        temp=TemperatureLogger(10),
        coords=CoordinateLogger(10),
    ),
)

System with 100 atoms, boundary CubicBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}[2.0 nm, 2.0 nm, 2.0 nm])

In [32]:
simulator = VelocityVerlet(
    dt=0.002u"ps",
    coupling=AndersenThermostat(temp, 1.0u"ps"),
)

VelocityVerlet{Quantity{Float64, 𝐓, Unitful.FreeUnits{(ps,), 𝐓, nothing}}, AndersenThermostat{Quantity{Float64, 𝚯, Unitful.FreeUnits{(K,), 𝚯, nothing}}, Quantity{Float64, 𝐓, Unitful.FreeUnits{(ps,), 𝐓, nothing}}}}(0.002 ps, AndersenThermostat{Quantity{Float64, 𝚯, Unitful.FreeUnits{(K,), 𝚯, nothing}}, Quantity{Float64, 𝐓, Unitful.FreeUnits{(ps,), 𝐓, nothing}}}(100.0 K, 1.0 ps), 1)

In [33]:
simulate!(sys, simulator, 1_000)

System with 100 atoms, boundary CubicBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}[2.0 nm, 2.0 nm, 2.0 nm])

In [16]:
Pkg.add("GLMakie")

   Resolving package versions...


   Installed TiffImages ─────────── v0.6.4
   Installed MiniQhull ──────────── v0.4.0
   Installed PNGFiles ───────────── v0.3.17
   Installed StableHashTraits ───── v0.3.1
   Installed ModernGL ───────────── v1.1.7
   Installed JpegTurbo ──────────── v0.1.2
   Installed Netpbm ─────────────── v1.1.0
   Installed AxisArrays ─────────── v0.4.6
   Installed GLMakie ────────────── v0.8.6


   Installed QhullMiniWrapper_jll ─ v1.0.0+1
   Installed TupleTools ─────────── v1.3.0
   Installed TriplotBase ────────── v0.1.0
   Installed ImageMetadata ──────── v0.9.8
   Installed MosaicViews ────────── v0.3.4
   Installed PaddedViews ────────── v0.5.12
   Installed GridLayoutBase ─────── v0.9.1


   Installed Imath_jll ──────────── v3.1.7+0


   Installed RangeArrays ────────── v0.3.2
   Installed OpenEXR_jll ────────── v3.1.4+0
   Installed MakieCore ──────────── v0.6.3
   Installed Packing ────────────── v0.5.0
   Installed FreeTypeAbstraction ── v0.10.0
   Installed ImageBase ──────────── v0.1.5
   Installed GeometryBasics ─────── v0.4.7


...

  ✓ Netpbm


  ✓ ImageIO


  ✓ Makie


  ✓ Brillouin → BrillouinMakieExt


  ✓ GLMakie
  54 dependencies successfully precompiled in 127 seconds. 506 already precompiled.


In [17]:
using GLMakie

### Let's visualize!

In [34]:
visualize(sys.loggers.coords, boundary, "../Simulations/atom_lennard_jones_100K.mp4")

"../Simulations/atom_lennard_jones_100K.mp4"

# Let us try at higher temperatures

In [35]:
temp = 300.0u"K"

300.0 K

In [36]:
velocities = [random_velocity(atom_mass, temp) for i in 1:n_atoms]

100-element Vector{SVector{3, Quantity{Float64, 𝐋 𝐓⁻¹, Unitful.FreeUnits{(nm, ps⁻¹), 𝐋 𝐓⁻¹, nothing}}}}:
 [-0.14328605260088761 nm ps⁻¹, -0.07045134810953785 nm ps⁻¹, 0.4091662187351717 nm ps⁻¹]
 [0.1782244712767378 nm ps⁻¹, -0.7252720860244865 nm ps⁻¹, 0.22431500689105519 nm ps⁻¹]
 [0.17631209215601484 nm ps⁻¹, 0.3011964751790961 nm ps⁻¹, 0.6987030585153833 nm ps⁻¹]
 [-0.041286270892538224 nm ps⁻¹, -0.06508478938228514 nm ps⁻¹, -0.2577835348409147 nm ps⁻¹]
 [-0.38763799845269375 nm ps⁻¹, -0.10980507077175464 nm ps⁻¹, -0.3551253524542351 nm ps⁻¹]
 [0.4368265161487977 nm ps⁻¹, -0.4921899220780395 nm ps⁻¹, -0.44367839957525346 nm ps⁻¹]
 [-0.7208722571392884 nm ps⁻¹, 0.6520105514380281 nm ps⁻¹, -0.1636119403174176 nm ps⁻¹]
 [-0.5267778173872996 nm ps⁻¹, -0.981256020335572 nm ps⁻¹, -0.7874035223687714 nm ps⁻¹]
 [-0.49994658133307895 nm ps⁻¹, 0.8869242008090592 nm ps⁻¹, -0.6728867300898289 nm ps⁻¹]
 [-0.5731850679250744 nm ps⁻¹, -0.6552102909362094 nm ps⁻¹, -0.43518474881725705 nm ps⁻¹]
 ⋮


In [37]:
simulator = VelocityVerlet(
    dt=0.002u"ps",
    coupling=AndersenThermostat(temp, 1.0u"ps"),
)

VelocityVerlet{Quantity{Float64, 𝐓, Unitful.FreeUnits{(ps,), 𝐓, nothing}}, AndersenThermostat{Quantity{Float64, 𝚯, Unitful.FreeUnits{(K,), 𝚯, nothing}}, Quantity{Float64, 𝐓, Unitful.FreeUnits{(ps,), 𝐓, nothing}}}}(0.002 ps, AndersenThermostat{Quantity{Float64, 𝚯, Unitful.FreeUnits{(K,), 𝚯, nothing}}, Quantity{Float64, 𝐓, Unitful.FreeUnits{(ps,), 𝐓, nothing}}}(300.0 K, 1.0 ps), 1)

In [40]:
simulate!(sys, simulator, 2_000)

System with 100 atoms, boundary CubicBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}[2.0 nm, 2.0 nm, 2.0 nm])

In [41]:
visualize(sys.loggers.coords, boundary, "../Simulations/atom_lennard_jones_150K.mp4")

"../Simulations/atom_lennard_jones_150K.mp4"

## Simulating a diatomic system

In [ ]:
coords = place_atoms(n_atoms ÷ 2, boundary; min_dist=0.3u"nm")
for i in 1:length(coords)
    push!(coords, coords[i] .+ [0.1, 0.0, 0.0]u"nm")
end

velocities = [velocity(atom_mass, temp) for i in 1:n_atoms]
#### Now we can use the built-in interaction list and bond types to place harmonic bonds between paired atoms.
bonds = InteractionList2Atoms(
    collect(1:(n_atoms ÷ 2)),           # First atom indices
    collect((1 + n_atoms ÷ 2):n_atoms), # Second atom indices
    repeat([""], n_atoms ÷ 2),          # Bond types
    [HarmonicBond(k=300_000.0u"kJ * mol^-1 * nm^-2", r0=0.1u"nm") for i in 1:(n_atoms ÷ 2)],
)

specific_inter_lists = (bonds,)
##### This time, we are also going to use a neighbor list to speed up the Lennard Jones calculation. We can use the built-in ```DistanceNeighborFinder```. The arguments are a 2D array of eligible interacting pairs, the number of steps between each update and the distance cutoff to be classed as a neighbor. Since the neighbor finder is run every 10 steps we should also use a cutoff for the interaction with a cutoff distance less than the neighbor list distance.
# All pairs apart from bonded pairs are eligible for non-bonded interactions
nb_matrix = trues(n_atoms, n_atoms)
for i in 1:(n_atoms ÷ 2)
    nb_matrix[i, i + (n_atoms ÷ 2)] = false
    nb_matrix[i + (n_atoms ÷ 2), i] = false
end

neighbor_finder = DistanceNeighborFinder(
    nb_matrix=nb_matrix,
    n_steps=10,
    dist_cutoff=1.5u"nm",
)

pairwise_inters = (LennardJones(nl_only=true, cutoff=DistanceCutoff(1.2u"nm")),)
sys = System(
    atoms=atoms,
    pairwise_inters=pairwise_inters,
    specific_inter_lists=specific_inter_lists,
    coords=coords,
    velocities=velocities,
    boundary=boundary,
    neighbor_finder=neighbor_finder,
    loggers=(
        temp=TemperatureLogger(10),
        coords=CoordinateLogger(10),
    ),
)

simulator = VelocityVerlet(
    dt=0.002u"ps",
    coupling=AndersenThermostat(temp, 1.0u"ps"),
)
simulate!(sys, simulator, 1_000)
#### This time when we view the trajectory we can add lines to show the bonds.
visualize(
    sys.loggers.coords,
    boundary,
    "sim_diatomic.mp4";
    connections=[(i, i + (n_atoms ÷ 2)) for i in 1:(n_atoms ÷ 2)],
)